In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
sys.path.append("/home/hb/python/")
sys.path.append("/home/hb/python/efficientnet_kincnn/code")
import kincnn
import kincnn2
import kincnn3
import kincnn4
import sys
sys.path.append('/home/hb/python/phospho/code/module')
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import sklearn
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import os 

def load_multiple_model(filedir, datatype):
    
    model_path_list = [f'{filedir}/{file}' for file in os.listdir(filedir) if "best" in file]
    model = []

    for model_path in model_path_list:
        print(model_path)
        _checkpoint = torch.load(model_path, map_location='cpu')
        _fold = _checkpoint['model']
        _fold.load_state_dict(_checkpoint['state_dict'])
        model.append(_fold)
    
    return model

def predict_model(model_name, df, matrix_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 1024
    matrix = []
    for i in df[matrix_name]:
        matrix.append(i)

    matrix = np.array(matrix).astype('float64')
    dataset = torch.utils.data.TensorDataset(torch.tensor(matrix))
    test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE)

    model = model_name.to(device)
    model.eval()
    preds_list = []
    probs_list = []
    with torch.no_grad():
        for matrix in tqdm(test_loader):
            matrix = torch.as_tensor(matrix[0], device=device, dtype=torch.float32)
            preds = model(matrix)
            probs_list.extend(preds.cpu().tolist())
            preds = torch.softmax(preds, dim=1)
            preds_list.extend(preds.cpu().tolist())

    return np.array(probs_list)

def ensemble_5fold(model_list, df, matrix_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    predict_list = []
    for model in model_list:
        prediction = predict_model(model_name=model, df=df, matrix_name=matrix_name)
        predict_list.append(prediction)
    ensemble = (predict_list[0] + predict_list[1] + predict_list[2] + predict_list[3] + predict_list[4])/len(predict_list)
    return ensemble

def ensemble(modelname, df):
    ensemble = ensemble_5fold(modelname, df, 'matrix')

    df['probs'] = ensemble
    df['answer'] = df['answer'].apply(lambda x: int(x)) 
    auc = sklearn.metrics.roc_auc_score(df['answer'], df[f'probs'])
    return auc

In [3]:
import re
def calculate_auc(time, day, filedir=None):

    df_test = pd.read_pickle("/home/hb/python/phospho/data/required/0308_final_train_test/test1.pkl")
    save_dir = f'/home/hb/python/efficientnet_kincnn/saved_model/{day}'

    if not filedir :
        filedir_list = [f'{save_dir}/DeepPP_kincnn{x}_{time}_bs1024_weight0' for x in range(1, 5)]
    else:
        filedir_list = [f'{filedir}']
    model_list = []
    auc_list = []
    for filedir in filedir_list:
        filedir_found = re.search('DeepPP_(.+?)_', filedir).group(1)
        model = load_multiple_model(filedir, filedir_found)
        model_list.append(model)
        auc = ensemble(model, df_test)
        auc_list.append(round(auc, 4))
    return model_list, auc_list, filedir_list

model_list, auc_list, filedir_list = calculate_auc('1847', '0519')

/home/hb/python/efficientnet_kincnn/saved_model/0519/DeepPP_kincnn1_1847_bs1024_weight0/3fold_best_model.pth
/home/hb/python/efficientnet_kincnn/saved_model/0519/DeepPP_kincnn1_1847_bs1024_weight0/0fold_best_model.pth
/home/hb/python/efficientnet_kincnn/saved_model/0519/DeepPP_kincnn1_1847_bs1024_weight0/1fold_best_model.pth
/home/hb/python/efficientnet_kincnn/saved_model/0519/DeepPP_kincnn1_1847_bs1024_weight0/4fold_best_model.pth
/home/hb/python/efficientnet_kincnn/saved_model/0519/DeepPP_kincnn1_1847_bs1024_weight0/2fold_best_model.pth


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
auc_list

[0.8426, 0.8438, 0.8438, 0.843]

In [6]:
import warnings

warnings.filterwarnings(action='ignore') 

# df = pd.read_excel('/home/hb/python/efficientnet_kincnn/model_info.xlsx', engine='openpyxl')
df_avg = pd.read_excel('model_info.xlsx', sheet_name=['AVG'])['AVG']
df = pd.read_excel('model_info.xlsx', sheet_name=['TOTAL'])['TOTAL']

for idx, model, auc, model_path in zip(range(len(model_list)), model_list, auc_list, filedir_list):
    _model = model[0]
    model_params_dict = {}
    model_params_dict['data'] = f'train{idx+1}'
    model_params_dict['AUC'] = auc
    model_params_dict['conv_stem_kernel_size'] = _model._conv_stem.kernel_size
    model_params_dict['conv_stem_stride_size'] = _model._conv_stem.stride
    model_params_dict['conv_stem_pooling'] = bool(_model._max_pooling)
    if model_params_dict['conv_stem_pooling']:
        model_params_dict['conv_stem_pooling_kernel_size'] = _model._max_pooling.kernel_size
    # model_params_dict[]
    blocks_num = len(_model._blocks)
    for i in range(blocks_num):
        model_params_dict[f'MBconv{i}_inp'] = _model._blocks[i]._depthwise_conv.in_channels
        model_params_dict[f'MBconv{i}_oup'] = _model._blocks[i]._project_conv.out_channels
        model_params_dict[f'MBconv{i}_kernel_size'] = _model._blocks[i]._depthwise_conv.kernel_size
        model_params_dict[f'MBconv{i}_stride'] = _model._blocks[i]._depthwise_conv.stride
    model_params_dict['last_features'] = _model._fc.in_features
    model_params_dict['model_path'] = '/'.join(model_path.split('/')[-2:])
    df = df.append(model_params_dict, ignore_index=True)

import statistics
model_avg_params_dict = model_params_dict.copy()
model_avg_params_dict['data'] = f'{len(df)-4}~{len(df)}avg'
model_avg_params_dict['AUC']= statistics.mean(auc_list)
df_avg = df_avg.append(model_avg_params_dict, ignore_index=True)

In [7]:
with pd.ExcelWriter('model_info.xlsx') as writer:
    df_avg.to_excel(writer, sheet_name='AVG', index=False)
    df.to_excel(writer, sheet_name='TOTAL', index=False)

df_avg = pd.read_excel('model_info.xlsx', sheet_name=['AVG'])['AVG']
df = pd.read_excel('model_info.xlsx', sheet_name=['TOTAL'])['TOTAL']

In [11]:
df_avg

,data,AUC,conv_stem_kernel_size,conv_stem_stride_size,conv_stem_pooling,conv_stem_pooling_kernel_size,MBconv0_inp,MBconv0_oup,MBconv0_kernel_size,MBconv0_stride,...,MBconv2_inp,MBconv2_oup,MBconv2_kernel_size,MBconv2_stride,last_features,model_path,MBconv3_inp,MBconv3_oup,MBconv3_kernel_size,MBconv3_stride
0,56~60avg,0.839600,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(1, 1)",...,32,64,"(3, 1)","(2, 1)",3960,0519/DeepPP_kincnn4_0001_bs1024_weight0,NaN,NaN,NaN,NaN
1,60~64avg,0.841550,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(1, 1)",...,32,64,"(3, 1)","(1, 1)",7920,0519/DeepPP_kincnn4_1400_bs1024_weight0,NaN,NaN,NaN,NaN
2,64~68avg,0.837350,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(1, 1)",...,32,64,"(3, 1)","(1, 1)",7920,0519/DeepPP_kincnn4_1511_bs1024_weight0,64.0,128.0,"(5, 5)","(1, 1)"
3,68~72avg,0.843225,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(1, 1)",...,32,64,"(3, 1)","(1, 1)",7920,0519/DeepPP_kincnn4_1736_bs1024_weight0,64.0,128.0,"(5, 3)","(1, 1)"
4,72~76avg,0.843300,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(1, 1)",...,32,64,"(3, 1)","(2, 1)",3960,0519/DeepPP_kincnn4_1847_bs1024_weight0,64.0,128.0,"(5, 3)","(1, 1)"


In [9]:
df

,data,AUC,conv_stem_kernel_size,conv_stem_stride_size,conv_stem_pooling,conv_stem_pooling_kernel_size,MBconv0_inp,MBconv0_oup,MBconv0_kernel_size,MBconv0_stride,...,MBconv13_kernel_size,MBconv13_stride,MBconv14_inp,MBconv14_oup,MBconv14_kernel_size,MBconv14_stride,MBconv15_inp,MBconv15_oup,MBconv15_kernel_size,MBconv15_stride
0,train1,0.8273,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(3, 1)","(1, 1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,train1,0.8202,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 5)","(2, 1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,train1,0.8243,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 3)","(2, 1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,train1,0.8262,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(2, 1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,train1,0.6549,"(3, 1)","(1, 1)",True,"(2, 1)",8,8,"(3, 3)","(1, 1)",...,"(5, 5)","(1, 1)",720.0,120.0,"(5, 5)","(1, 1)",720.0,200.0,"(3, 3)","(1, 1)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,train4,0.8463,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(1, 1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,train1,0.8426,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(1, 1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,train2,0.8438,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(1, 1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,train3,0.8438,"(3, 1)","(1, 1)",True,"(2, 1)",8,16,"(5, 1)","(1, 1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_test['st/y'] = df_test['subseq'].apply(lambda x: 'y' if x[7]=='Y' else 'st')

NameError: name 'df_test' is not defined

In [ ]:
df_test_st = df_test[df_test['st/y']=='st']
df_test_y = df_test[df_test['st/y']=='y']

In [ ]:
df_test_st.head()

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,subseq11,pair,st/y
0,PLK4,O00444,HAND1,Q64279,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,KERRRTESINSAFAE,S109,12518702.0,human,mouse,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSMUSP00000124951,RRRTESINSAF,O00444_RRRTESINSAF,st
1,PLK4,O00444,PLK4,O00444,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,MPHEKHYTLCGTPNY,T170,480636.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000427568,HEKHYTLCGTP,O00444_HEKHYTLCGTP,st
2,YSK1,O00506,PDCD10,Q9BUL8,FTKLDRIGKGSFGEVYKGIDNHTKEVVAIKIIDLEEAEDEIEDIQQ...,VNLSAAQTLRAAFIK,T43,7452901.0,human,human,1,263.0,"[[[1.1606729892090855, 0.9617507091463667, 1.0...",ENSP00000418160,LSAAQTLRAAF,O00506_LSAAQTLRAAF,st
3,IKKB,O14920,IKKG,Q9Y6K9,0EMKERLGTGGFGNVIRWHNQETGEQIAIKQCRQELSPRNRERWCL...,LRDAIRQSNQILRER,S68,1226900.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000484275,DAIRQSNQILR,O14920_DAIRQSNQILR,st
4,IKKB,O14920,NFkB-p65,Q04206,0EMKERLGTGGFGNVIRWHNQETGEQIAIKQCRQELSPRNRERWCL...,SGDEDFSSIADMDFS,S536,447800.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000433616,DEDFSSIADMD,O14920_DEDFSSIADMD,st


In [ ]:
# test_pair = set(df_test['pair'])
test_pair = set(df_test_st['pair'])

In [ ]:
auc = ensemble(model, df_test_st)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_572123/976407143.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['probs'] = ensemble
/tmp/ipykernel_572123/976407143.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = df['answer'].apply(lambda x: int(x))


In [ ]:
auc

0.7115552487870236

In [ ]:
nwk = pd.read_pickle("/home/hb/python/phospho/data/nwk/nwk_mapping_by_uniprot.pkl")

In [ ]:
nwk['pair'] = nwk['kin_id'] + '_' + nwk['subseq']
netphorest_score_dt = {pair: netphorest_score  for pair, netphorest_score in zip(nwk['pair'], nwk['netphorest_score'])}
networkin_score_dt = {pair: networkin_score  for pair, networkin_score in zip(nwk['pair'], nwk['networkin_score'])}

In [ ]:
from collections import namedtuple
score_all_tp = namedtuple('score_all_tp', ['networkin', 'netphorest'])

In [ ]:
nwk_test = nwk[nwk['pair'].isin(df_test['pair'])]

In [ ]:
nwk_test.head(2)

,kinase,kin_id,substrate,ensemble_id,kinseq,subseq,sub_mod_rsd,netphorest_score,networkin_score,pair
52,CDK1,P06493,AAK1,ENSP00000385181,NaN,IQPALTPRKRA,T389,0.3539,0.6944,P06493_IQPALTPRKRA
161,IKKA,O15111,AATF,ENSP00000225402,NaN,IERKTSSLDPN,S477,0.0206,1.0765,O15111_IERKTSSLDPN


In [ ]:
nwk = pd.read_pickle("/home/hb/python/phospho/data/nwk/nwk_mapping_by_uniprot.pkl", )
nwk['kin_id'] = nwk['kin_id'].apply(lambda x: x.upper()) 
nwk['subseq'] = nwk['subseq'].apply(lambda x: x.upper()) 
nwk_pair = nwk['kin_id'] + '_' + nwk['subseq']
nwk.insert(0, 'pair', nwk_pair)
nwk.head()

,pair,kinase,kin_id,substrate,ensemble_id,kinseq,subseq,sub_mod_rsd,netphorest_score,networkin_score
0,P17252_REEILSEMKKV,PKCA,P17252,A1CF,ENSP00000363105,NaN,REEILSEMKKV,S154,0.1576,1.0071
1,P05771_REEILSEMKKV,PKCB,P05771,A1CF,ENSP00000363105,NaN,REEILSEMKKV,S154,0.1576,1.3889
2,P17252_TKGHLSNRAII,PKCA,P17252,A1CF,ENSP00000363105,NaN,TKGHLSNRAII,S369,0.3374,1.7569
3,P05771_TKGHLSNRAII,PKCB,P05771,A1CF,ENSP00000363105,NaN,TKGHLSNRAII,S369,0.3374,2.4948
4,Q05655_TKGHLSNRAII,PKCD,Q05655,A1CF,ENSP00000363105,NaN,TKGHLSNRAII,S369,0.3374,0.5110


In [ ]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
nwk_log = np.log(nwk.networkin_score)
nwk_log = np.array(nwk_log).reshape(-1, 1)
nwk_log = mm.fit_transform(nwk_log)

In [ ]:
nwk['log_scale'] = nwk_log

In [ ]:
nwk.tail()

,pair,kinase,kin_id,substrate,ensemble_id,kinseq,subseq,sub_mod_rsd,netphorest_score,networkin_score,log_scale
177231,P06493_SSEEDSPIKSD,CDK1,P06493,ZZZ3,ENSP00000359837,NaN,SSEEDSPIKSD,S135,0.3225,0.6944,0.053608
177232,Q00526_SSEEDSPIKSD,CDK3,Q00526,ZZZ3,ENSP00000359837,NaN,SSEEDSPIKSD,S135,0.3225,0.5236,0.007500
177233,P27361_SSEEDSPIKSD,MAPK3,P27361,ZZZ3,ENSP00000359837,NaN,SSEEDSPIKSD,S135,0.1783,1.4275,0.171303
177234,Q13153_AKESASQHITE,PAK1,Q13153,ZZZ3,ENSP00000359837,NaN,AKESASQHITE,S474,0.0438,0.7524,0.066710
177235,P05771_AKESASQHITE,PKCB,P05771,ZZZ3,ENSP00000359837,NaN,AKESASQHITE,S474,0.0921,0.8358,0.083879


In [ ]:
# pair 컬럼있고, 컬럼명정리됐고, uniprot_id 맵핑했고, networkin_Score도 마무리됨
nwk.to_pickle("/home/hb/python/phospho/data/nwk/0218_nwk_f.pkl")

In [ ]:
nwk_test = nwk[nwk['pair'].isin(df_test['pair'])]

In [ ]:
nwk_score = {pair: score_all_tp(nwk_s, npr_s) for pair, nwk_s, npr_s in zip(nwk_test['pair'], nwk_test['log_scale'], nwk_test['netphorest_score'])}

In [ ]:
df_test['nwk'] = df_test['pair'].apply(lambda x: nwk_score[x].networkin if x in nwk_score.keys() else np.nan)

In [ ]:
df_test['npr'] = df_test['pair'].apply(lambda x: nwk_score[x].netphorest if x in nwk_score.keys() else np.nan)

In [ ]:
df_test.dropna(subset=['nwk', 'npr'], inplace=True)

In [ ]:
auc_ppsp = sklearn.metrics.roc_auc_score(df_test['answer'], df_test[f'probs'])
auc_npf = sklearn.metrics.roc_auc_score(df_test['answer'], df_test[f'npr'])
auc_nwk = sklearn.metrics.roc_auc_score(df_test['answer'], df_test[f'nwk'])

In [ ]:
auc_ppsp, auc_npf, auc_nwk

(0.7102616656134633, 0.4866818272524695, 0.7180266155585857)

In [ ]:
df_test.answer.value_counts()

1    2248
0     214
Name: answer, dtype: int64

In [ ]:
len(df_test)

2462

In [ ]:
st_subseq = [x for x in df_test['subseq'] if (x[7]=='S') or (x[7]=='T')]

In [ ]:
df_test_st = df_test[df_test['subseq'].isin(st_subseq)]
df_test_st.reset_index(inplace=True, drop=True)

In [ ]:
# # # pre
# df_test1 = pd.read_pickle("/home/hb/python/phospho/data/0111_new_data/train_test/test1.pickle")
# df_test2 = pd.read_pickle("/home/hb/python/phospho/data/0111_new_data/train_test/test2.pickle")
# df_test3 = pd.read_pickle("/home/hb/python/phospho/data/0111_new_data/train_test/test3.pickle")
# # df_test1.tail()

In [ ]:
# df_test1 = pd.read_pickle("/home/hb/python/phospho/data/0119_nwk_matching/test1.pickle")
# df_test2 = pd.read_pickle("/home/hb/python/phospho/data/0119_nwk_matching/test2.pickle")
# df_test3 = pd.read_pickle("/home/hb/python/phospho/data/0119_nwk_matching/test3.pickle")
# df_test1.tail()

In [ ]:
auc_dict = {}
for i in range(0, 5):
    auc_dict[f'auc{i}'] = ensemble(globals()[f'model_list_DeepPP_w{i}syt1'], df_test)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
auc_dict

{'auc0': 0.878752645750757,
 'auc1': 0.8745278989052743,
 'auc2': 0.8725035798506958,
 'auc3': 0.8748872810946196,
 'auc4': 0.8811933781395609}

In [ ]:
auc = ensemble(model_list_w0syt1, df_test)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
auc

0.8725035798506958

In [ ]:
df = pd.read_pickle("/home/hb/python/phospho/data/0111_new_data/train_test/train1.pickle")
df.head()

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
0,RIPK3,Q9Y572,NaN,NaN,0ENQELVGKGGFGTVFRAQHRKWGYDVAVKIVNSKAISREVKAMAS...,PPPLAPQSPQGGVMG,NaN,NaN,NaN,human,0,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000478927,RIPK
1,ABL,P00519,NaN,NaN,ITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEA...,LKKTDRIYAMKVVKK,NaN,NaN,NaN,rat,0,NaN,"[[[1.1984151792931996, 1.0140984589384923, 1.0...",ENSRNOP00000012924,Abl
2,CDK1,P06493,NaN,NaN,YTKIEKIGEGTYGVVYKGRHKTTGQVVAMKKIRLESEEEGVPSTAI...,SGEEECRSQPRSISE,NaN,NaN,NaN,mouse,0,NaN,"[[[1.0232665395472176, 0.981179362242806, 0.86...",ENSMUSP00000137933,CDK
3,CK1A,P48729,PHLPP,O60346,YKLVRKIGSGSFGDIYLAINITNGEEVAVKLESQKARHPQLLYESK...,GSKGLWDSLSVEEAV,S1379,22769302.0,human,human,1,263.0,"[[[0.981179362242806, 1.0232665395472176, 1.01...",ENSP00000467279,CK1
4,CK2A1,P68400,SIRT1,Q96EB6,YQLVRKLGRGKYSEVFEAINITNNEKVVVKILKPVKKKKIKREIKI...,FHGAEVYSDSEDDVL,S659,6855923.0,human,human,1,263.0,"[[[1.2104594520812995, 1.0639623447280337, 0.9...",ENSP00000384063,CK2


In [ ]:
tr_pair = df['kin_id'] + '_' + df['subseq']
tr_pair = set(tr_pair)

In [ ]:
tt_pair = df_test1['kin_id'] + '_' + df_test1['subseq']
tt_pair = set(tt_pair)
df_test1.head()

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family,probs
0,FAK,Q05397,FAK,Q05397,IELGRCIGEGQFGDVHQGIYMSPENPALAVAIKTCKNCTSDSVREK...,IIDEEDTYTMPSTRD,Y407,447888.0,human,human,1,263.0,"[[[1.3951472984698037, 1.3951472984698037, 0.8...",ENSP00000431023,Fak,0.632388
1,ABL,P00519,Rb,P06400,ITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEA...,RIPGGNIYISPLKSP,Y805,456752.0,human,human,1,263.0,"[[[0.9656054162575665, 1.3951472984698037, 0.7...",ENSP00000496005,Abl,0.639248
2,MELK,Q14680,MELK,Q14680,YELHETIGTGGFAKVKLACHILTGEMVAIKIMDKNTLGSDLPRIKT...,SPERRCRSVELDLNQ,S505,2861409.0,human,human,1,263.0,"[[[1.0232665395472176, 0.8860339595928756, 0.8...",ENSP00000443550,CAMKL,0.774461
3,PKACA,P17612,EMSY,Q7Z589,FERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQEHTL...,KPRKRRRTNSSSSSP,T207,1210942.0,human,human,1,263.0,"[[[0.9762857097579093, 0.8702280284582515, 0.9...",ENSP00000436307,PKA,0.960219
4,SRC,P12931,tensin 3,Q68CZ2,0RLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQ...,QDTSKFWYKADISRE,Y1173,9824600.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000407464,Src,0.472669


In [ ]:
df_test1['kinase'] = df_test1['kinase'].apply(lambda x: x.upper()) 
df_test1['subseq11'] = df_test1['subseq'].apply(lambda x: x[2:-2])
df_test1['pair'] = df_test1['kinase'] + '_' + df_test1['subseq11']
df_test1.head(3)

,pair,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,subseq11,networkin_score,netphorest_score,string_score,probs
0,PLK4_RRRTESINSAF,PLK4,O00444,HAND1,Q64279,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,KERRRTESINSAFAE,S109,12518702.0,human,mouse,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSMUSP00000124951,RRRTESINSAF,1.5492,0.0249,0.563,0.909146
1,PLK4_HEKHYTLCGTP,PLK4,O00444,PLK4,O00444,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,MPHEKHYTLCGTPNY,T170,480636.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000427568,HEKHYTLCGTP,24.4009,0.0965,1.000,0.811036
2,YSK1_LSAAQTLRAAF,YSK1,O00506,PDCD10,Q9BUL8,FTKLDRIGKGSFGEVYKGIDNHTKEVVAIKIIDLEEAEDEIEDIQQ...,VNLSAAQTLRAAFIK,T43,7452901.0,human,human,1,263.0,"[[[1.1606729892090855, 0.9617507091463667, 1.0...",ENSP00000418160,LSAAQTLRAAF,36.6294,0.1918,0.999,0.491396


In [ ]:
nwk = pd.read_pickle("/home/hb/python/phospho/data/benchmark_data/nwk.pickle", )
nwk['kinase'] = nwk['kinase'].apply(lambda x: x.upper()) 
nwk['sequence'] = nwk['sequence'].apply(lambda x: x.upper()) 
nwk_pair = nwk['kinase'] + '_' + nwk['sequence']
nwk.insert(0, 'pair', nwk_pair)
nwk.head()

,pair,substrate,substrate_id,#substrate,position,id,networkin_score,tree,netphorest_group,netphorest_score,string_identifier,string_score,substrate_name,sequence,string_path,kinase
0,PKCA_REEILSEMKKV,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCalpha,1.0071,KIN,PKC_group,0.1576,ENSP00000282641,0.1738,A1CF,REEILSEMKKV,"ENSP00000284384, 0.7304 ENSP00000385269, 0.675...",PKCA
1,PKCB_REEILSEMKKV,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCbeta,1.3889,KIN,PKC_group,0.1576,ENSP00000282641,0.1232,A1CF,REEILSEMKKV,"ENSP00000305355, 0.7328 ENSP00000348986, 0.627...",PKCB
2,PKCZ_REEILSEMKKV,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCzeta,0.5163,KIN,PKC_group,0.1576,ENSP00000282641,0.1685,A1CF,REEILSEMKKV,"ENSP00000367830, 0.6584 ENSP00000226574, 0.727...",PKCZ
3,PKCA_TKGHLSNRAII,A1CF,ENSP00000363105,A1CF (ENSP00000363105),369,PKCalpha,1.7569,KIN,PKC_group,0.3374,ENSP00000282641,0.1738,A1CF,TKGHLSNRAII,"ENSP00000284384, 0.7304 ENSP00000385269, 0.675...",PKCA
4,PKCB_TKGHLSNRAII,A1CF,ENSP00000363105,A1CF (ENSP00000363105),369,PKCbeta,2.4948,KIN,PKC_group,0.3374,ENSP00000282641,0.1232,A1CF,TKGHLSNRAII,"ENSP00000305355, 0.7328 ENSP00000348986, 0.627...",PKCB


In [ ]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
nwk_log = np.log(nwk.networkin_score)
nwk_log = np.array(nwk_log).reshape(-1, 1)
nwk_log = mm.fit_transform(nwk_log)

In [ ]:
nwk['log_scale'] = nwk_log

In [ ]:
nwk_scale = np.array(nwk.networkin_score)
nwk_scale =  nwk_scale.reshape(-1, 1)

In [ ]:
nwk_scale = mm.fit_transform(nwk_scale)

In [ ]:
nwk['nwk_scale'] = nwk_scale

In [ ]:
nwk.head(3)

,pair,substrate,substrate_id,#substrate,position,id,networkin_score,tree,netphorest_group,netphorest_score,string_identifier,string_score,substrate_name,sequence,string_path,kinase,log_scale,nwk_scale
0,PKCA_REEILSEMKKV,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCalpha,1.0071,KIN,PKC_group,0.1576,ENSP00000282641,0.1738,A1CF,REEILSEMKKV,"ENSP00000284384, 0.7304 ENSP00000385269, 0.675...",PKCA,0.114328,0.002227
1,PKCB_REEILSEMKKV,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCbeta,1.3889,KIN,PKC_group,0.1576,ENSP00000282641,0.1232,A1CF,REEILSEMKKV,"ENSP00000305355, 0.7328 ENSP00000348986, 0.627...",PKCB,0.166826,0.003904
2,PKCZ_REEILSEMKKV,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCzeta,0.5163,KIN,PKC_group,0.1576,ENSP00000282641,0.1685,A1CF,REEILSEMKKV,"ENSP00000367830, 0.6584 ENSP00000226574, 0.727...",PKCZ,0.005207,0.000071


In [ ]:
scores = namedtuple('scores_nt', ['nwk_log', 'nwk_scale' 'netphorest_score', 'string_score'])

In [ ]:
k = scores(3, 3, 3)

In [ ]:
k

scores_nt(nwk_log=3, nwk_scalenetphorest_score=3, string_score=3)

In [ ]:
from collections import namedtuple
# import dill
scores_nt = namedtuple('scores_nt', ['nwk_log', 'nwk_scale', 'netphorest_score', 'string_score'])
nwk_score = {pair: scores_nt(nwk_log, nwk_scale, npr_score, string_score) 
                for pair, nwk_log, nwk_scale, npr_score, string_score in 
            zip(nwk.pair, nwk.log_scale, nwk.nwk_scale, nwk.netphorest_score, nwk.string_score)}

In [ ]:
ppsp_nwk_log_score = [nwk_score[x].nwk_log if x in nwk_score.keys() else np.NaN for x in df_test1.pair ]
ppsp_nwk_score = [nwk_score[x].nwk_scale if x in nwk_score.keys() else np.NaN for x in df_test1.pair ]
ppsp_npr_score = [nwk_score[x].netphorest_score if x in nwk_score.keys() else np.NaN for x in df_test1.pair ]
ppsp_str_score = [nwk_score[x].string_score if x in nwk_score.keys() else np.NaN for x in df_test1.pair ]

In [ ]:
df_test1['nwk_scale'] = ppsp_nwk_score
df_test1['nwk_log'] = ppsp_nwk_log_score
df_test1['netphorest_score'] = ppsp_npr_score
df_test1['string_score'] = ppsp_str_score
df_in_nwk = df_test1.dropna(subset=['networkin_score'])

In [ ]:
df_in_nwk.reset_index(inplace=True, drop=True)
len(df_in_nwk)

3498

In [141]:
df_in_nwk.head()

,pair,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,...,length,matrix,ensemble_id,subseq11,networkin_score,netphorest_score,string_score,probs,nwk_scale,nwk_log
0,PLK4_RRRTESINSAF,PLK4,O00444,HAND1,Q64279,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,KERRRTESINSAFAE,S109,12518702.0,human,...,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSMUSP00000124951,RRRTESINSAF,1.5492,0.0249,0.563,0.909146,0.004609,0.184665
1,PLK4_HEKHYTLCGTP,PLK4,O00444,PLK4,O00444,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,MPHEKHYTLCGTPNY,T170,480636.0,human,...,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000427568,HEKHYTLCGTP,24.4009,0.0965,1.000,0.811036,0.104993,0.634921
2,YSK1_LSAAQTLRAAF,YSK1,O00506,PDCD10,Q9BUL8,FTKLDRIGKGSFGEVYKGIDNHTKEVVAIKIIDLEEAEDEIEDIQQ...,VNLSAAQTLRAAFIK,T43,7452901.0,human,...,263.0,"[[[1.1606729892090855, 0.9617507091463667, 1.0...",ENSP00000418160,LSAAQTLRAAF,36.6294,0.1918,0.999,0.491396,0.158711,0.701267
3,IKKB_DAIRQSNQILR,IKKB,O14920,IKKG,Q9Y6K9,0EMKERLGTGGFGNVIRWHNQETGEQIAIKQCRQELSPRNRERWCL...,LRDAIRQSNQILRER,S68,1226900.0,human,...,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000484275,DAIRQSNQILR,6.0689,0.0344,0.999,0.372514,0.024463,0.407669
4,IKKB_DEDFSSIADMD,IKKB,O14920,NFkB-p65,Q04206,0EMKERLGTGGFGNVIRWHNQETGEQIAIKQCRQELSPRNRERWCL...,SGDEDFSSIADMDFS,S536,447800.0,human,...,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000433616,DEDFSSIADMD,2.5318,0.0293,0.999,0.676424,0.008925,0.264887


In [68]:
auc_ppsp = sklearn.metrics.roc_auc_score(df_in_nwk['answer'], df_in_nwk[f'probs'])
auc_npf = sklearn.metrics.roc_auc_score(df_in_nwk['answer'], df_in_nwk[f'netphorest_score'])

In [143]:
auc_nwk_log = sklearn.metrics.roc_auc_score(df_in_nwk['answer'], df_in_nwk[f'nwk_log'])
auc_nwk_log_scale = sklearn.metrics.roc_auc_score(df_in_nwk['answer'], df_in_nwk[f'nwk_scale'])


In [144]:
auc_nwk_log

0.7242918609208568

In [145]:
auc_nwk_log_scale

0.7242918609208568

In [69]:
auc_ppsp

0.6772830325957269

In [70]:
auc_npf

0.500201598496348

In [38]:
df_in_nwk.networkin_score.describe()

count    581.000000
mean       5.584464
std        7.408278
min        0.505900
25%        1.174900
50%        2.406200
75%        7.063900
max       40.855200
Name: networkin_score, dtype: float64

In [39]:
np.log(df_in_nwk.networkin_score).describe()

count    581.000000
mean       1.026915
std        1.157722
min       -0.681416
25%        0.161183
50%        0.878049
75%        1.954997
max        3.710034
Name: networkin_score, dtype: float64

In [25]:
df_test1_sub = df_test1.ensemble_id.unique()
df_test1_kin = df_test1.kinase.unique()
df_test1_pair = set(df_test1.kinase + '_' + df_test1.subseq)
df_test1_pair

{'AURB_DLTYGNRYKNVKLPD',
 'ERK2_KVYFLPITPHYVTQV',
 'SRC_PSKLKRSYSSPDITQ',
 'SRC_GLTTSQPSPAHYSYP',
 'IKKA_SQRQRSTSTPNVHMV',
 'CDK4_ETLPISSTPSKSVLP',
 'AURB_KRQQARKTASVLSKD',
 'ERK2_ELGSQEGYVPKNFID',
 'P38D_TAADMYLSPVRSPKK',
 'CK2A1_HAATRPDYISWGTQE',
 'PAK5_QASSTPLSPTRITRL',
 'INSR_SEELDENYVPMNPNS',
 'CHK1_EVVGGTDSSMDVFHL',
 'CHK2_KGKKFLQYNRLQLSR',
 'PKACA_RARRGSSSDEEGGPK',
 'PKACA_SSRWSKDYDVCVCHS',
 'AURB_LYNEYIVYDIAQVNL',
 'EGFR_ETRICKIYDSPCLPE',
 'AMPKA1_AQVSRRISFSASHRL',
 'YES_GLDPVEVYESLPEEL',
 'CDK6_VHPATPISPGRASGM',
 'MST1_VEFQRAQSLLSTDRE',
 'CDK1_EKASLQKSISITSAL',
 'GRK6_GQISAGYSPVIDCHT',
 'CHK1_AARKSAPSTGGVKKP',
 'CDK7_GEMDTTSYDVSVLNS',
 'AMPKA1_PVRMRRNSFTPLSSS',
 'ERK1_SSSNRPFTPPTSTGG',
 'ERK2_GSPRTPVSPVKFSPG',
 'EPHA4_VNGHLDSYEKVTQLP',
 'JNK1_FITPTGHSLKSLDLV',
 'SRC_GRYHRGGYRPVWNRR',
 'ERK2_VSRVPYPSPTCVKSE',
 'ERK2_LLDACGFYWGPLSVH',
 'CDK1_PGTDDVFTPAPSDSP',
 'PRKD1_DNVRYGISNIDTTIE',
 'PAK2_ELDELMASLSDFKIQ',
 'ERK2_DEHSSDPYHSGYEMP',
 'SRC_SAEEAPLYSKVTPRA',
 'CHK1_MLKRRNVSSFPDDA

In [82]:
nwk = pd.read_pickle("/home/hb/python/phospho/data/benchmark_data/nwk.pickle")
nwk['sequence'] = nwk['sequence'].apply(lambda x: x.upper())
nwk_sub = nwk.substrate_id.unique()
nwk_kin = nwk.kinase.unique()
nwk_pair = nwk['kinase'] + '_' + nwk['sequence']
nwk['pair'] = nwk_pair

In [84]:
nwk

,substrate,substrate_id,#substrate,position,id,networkin_score,tree,netphorest_group,netphorest_score,string_identifier,string_score,substrate_name,sequence,string_path,kinase,pair
0,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCalpha,1.0071,KIN,PKC_group,0.1576,ENSP00000282641,0.1738,A1CF,REEILSEMKKV,"ENSP00000284384, 0.7304 ENSP00000385269, 0.675...",PKCA,PKCA_REEILSEMKKV
1,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCbeta,1.3889,KIN,PKC_group,0.1576,ENSP00000282641,0.1232,A1CF,REEILSEMKKV,"ENSP00000305355, 0.7328 ENSP00000348986, 0.627...",PKCB,PKCB_REEILSEMKKV
2,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCzeta,0.5163,KIN,PKC_group,0.1576,ENSP00000282641,0.1685,A1CF,REEILSEMKKV,"ENSP00000367830, 0.6584 ENSP00000226574, 0.727...",PKCZ,PKCZ_REEILSEMKKV
3,A1CF,ENSP00000363105,A1CF (ENSP00000363105),369,PKCalpha,1.7569,KIN,PKC_group,0.3374,ENSP00000282641,0.1738,A1CF,TKGHLSNRAII,"ENSP00000284384, 0.7304 ENSP00000385269, 0.675...",PKCA,PKCA_TKGHLSNRAII
4,A1CF,ENSP00000363105,A1CF (ENSP00000363105),369,PKCbeta,2.4948,KIN,PKC_group,0.3374,ENSP00000282641,0.1232,A1CF,TKGHLSNRAII,"ENSP00000305355, 0.7328 ENSP00000348986, 0.627...",PKCB,PKCB_TKGHLSNRAII
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459743,ZZZ3,ENSP00000359837,ZZZ3 (ENSP00000359837),135,CK1alpha,1.3564,KIN,CK1_group,0.1338,ENSP00000359837,0.1079,ZZZ3,SSEEDSPIKSD,"ENSP00000261798, 0.54 ENSP00000363559, 0.2776 ...",CK1A,CK1A_SSEEDSPIKSD
459744,ZZZ3,ENSP00000359837,ZZZ3 (ENSP00000359837),135,MAPK3,1.4275,KIN,MAPK3_MAPK1_MAPK7_NLK_group,0.1783,ENSP00000359837,0.1328,ZZZ3,SSEEDSPIKSD,"ENSP00000263025, 0.54 ENSP00000363559, 0.3416 ...",MAPK3,MAPK3_SSEEDSPIKSD
459745,ZZZ3,ENSP00000359837,ZZZ3 (ENSP00000359837),474,CK1alpha,1.3564,KIN,CK1_group,0.1249,ENSP00000359837,0.1079,ZZZ3,AKESASQHITE,"ENSP00000261798, 0.54 ENSP00000363559, 0.2776 ...",CK1A,CK1A_AKESASQHITE
459746,ZZZ3,ENSP00000359837,ZZZ3 (ENSP00000359837),474,PAK1,0.7524,KIN,PAK_group,0.0438,ENSP00000359837,0.1020,ZZZ3,AKESASQHITE,"ENSP00000278568, 0.54 ENSP00000363559, 0.2624 ...",PAK1,PAK1_AKESASQHITE


In [86]:
tr = pd.read_pickle("/home/hb/python/phospho/data/0119_nwk_matching/train1.pickle")
tr.tail()

,pair,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,subseq11
22915,TRKA_PVKDPYVMFPQ,TRKA,P04629,NaN,NaN,00LKWELGEGAFGKVFLAECHNLPEQDKMLVAVKALKEASESARQD...,SEPVKDPYVMFPQNA,NaN,NaN,NaN,NaN,0,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000472989,PVKDPYVMFPQ
22916,ULK2_GDDWDTDPDFV,ULK2,Q8IYT8,NaN,NaN,YSKRDLVGHGAFAVVFRGRHRQTDWEVAIKSINKKNLSKSQILLGK...,TQGDDWDTDPDFVND,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0273678027634894, 1.003004504503377, 0.98...",ENSP00000419614,GDDWDTDPDFV
22917,JNK2_SSLSPSPASSI,JNK2,P45984,NFAT4,Q12968,YQQLKPIGSGAQGIVCAAFDTVLGINVAVKKLSRPFQNQTHAKRAY...,RESSLSPSPASSISS,S165,448858.0,human,human,1,263.0,"[[[1.0618365465453596, 0.8650222931107413, 1.0...",ENSP00000457441,SSLSPSPASSI
22918,HGK_RDKYKTLRQIR,HGK,O95819,Moesin,P26038,FELVEVVGNGTYGQVYKGRHVKTGQLAAIKVMDVTEDEEEEIKLEI...,LGRDKYKTLRQIRQG,T558,447706.0,human,human,1,263.0,"[[[1.0963648220808169, 1.0191816486174081, 0.9...",ENSP00000513134,RDKYKTLRQIR
22919,ERK2_GDADMYDLPKK,ERK2,P28482,NaN,NaN,YTNLSYIGEGAYGMVCSAYDNVNKVRVAIKKISPFEHQTYCQRTLR...,TRGDADMYDLPKKED,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0273678027634894, 1.0618365465453596, 0.9...",ENSP00000507059,GDADMYDLPKK


In [85]:
tt = pd.read_pickle("/home/hb/python/phospho/data/0111_new_data/train_test/test1.pickle")
tt

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
0,FAK,Q05397,FAK,Q05397,IELGRCIGEGQFGDVHQGIYMSPENPALAVAIKTCKNCTSDSVREK...,IIDEEDTYTMPSTRD,Y407,447888.0,human,human,1,263.0,"[[[1.3951472984698037, 1.3951472984698037, 0.8...",ENSP00000431023,Fak
1,ABL,P00519,Rb,P06400,ITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEA...,RIPGGNIYISPLKSP,Y805,456752.0,human,human,1,263.0,"[[[0.9656054162575665, 1.3951472984698037, 0.7...",ENSP00000496005,Abl
2,MELK,Q14680,MELK,Q14680,YELHETIGTGGFAKVKLACHILTGEMVAIKIMDKNTLGSDLPRIKT...,SPERRCRSVELDLNQ,S505,2861409.0,human,human,1,263.0,"[[[1.0232665395472176, 0.8860339595928756, 0.8...",ENSP00000443550,CAMKL
3,PKACA,P17612,EMSY,Q7Z589,FERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQEHTL...,KPRKRRRTNSSSSSP,T207,1210942.0,human,human,1,263.0,"[[[0.9762857097579093, 0.8702280284582515, 0.9...",ENSP00000436307,PKA
4,SRC,P12931,tensin 3,Q68CZ2,0RLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQ...,QDTSKFWYKADISRE,Y1173,9824600.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000407464,Src
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,AMPKA1,Q13131,NaN,NaN,00LGDTLGVGTFGKVKVGKHELTGHKVAVKILNRQKIRSLDVGKIR...,AQVSRRISFSASHRL,NaN,NaN,NaN,human,0,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000434688,CAMKL
3996,KSR2,Q6VAB6,NaN,NaN,LEIGELIGKGRFGQVYHGRWHGEVAIRLIDERDNEDQLKAFKREVM...,EAETLAETEPERHLG,NaN,NaN,NaN,human,0,NaN,"[[[0.9841273200552851, 1.3008267189517242, 0.9...",ENSP00000429729,RAF
3997,MAPKAPK2,P49137,NaN,NaN,000SQVLGLGINGKVLQIFNKRTQEKFALKMLQDCPKARREVELHW...,LVDSVAKTMDAGCKP,NaN,NaN,NaN,human,0,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000486923,MAPKAPK
3998,PKACA,P17612,NaN,NaN,FERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQEHTL...,EPEPENDYEDVEEMD,NaN,NaN,NaN,human,0,NaN,"[[[0.8798533791446438, 0.8702280284582515, 0.8...",ENSP00000419614,PKA


In [89]:
tt = pd.read_pickle("/home/hb/python/phospho/data/after1227_new_data/syt_false_dataset/after0214_train_test/syt1_test.pkl")

In [93]:
tt.reset_index(drop=True, inplace=True)
tt['kinase'] = tt['kinase'].apply(lambda x: x.upper())
tt['pair'] = tt['kinase'] + '_' + tt['subseq']

In [94]:
nwk[nwk['pair'].isin(tt['pair'])]

,substrate,substrate_id,#substrate,position,id,networkin_score,tree,netphorest_group,netphorest_score,string_identifier,string_score,substrate_name,sequence,string_path,kinase,pair


In [79]:
input_data = (tt.ensemble_id + ' ' + tt.subseq).tolist()
input_data = [x for x in input_data if '_' not in x]

In [87]:
# print('\n'.join(input_data))

In [88]:
# print('\n'.join((tt.ensemble_id + ' ' + tt.subseq).tolist()))

In [34]:
nwk[nwk['pair'].isin(df_pair)]

,substrate,substrate_id,#substrate,position,id,networkin_score,tree,netphorest_group,netphorest_score,string_identifier,string_score,substrate_name,sequence,string_path,kinase,pair


In [27]:
common_pair = df_test1_pair.intersection(nwk_pair)
len(common_pair)

0

In [30]:
df = pd.read_pickle("/home/hb/python/phospho/data/after1227_new_data/syt_false_dataset/1228_syt1_mtx_ensembl.pickle")
df.head()
df['kinase'] = df['kinase'].apply(lambda x: x.upper())
df_pair = df['kinase'] + '_' + df['subseq']


0        BMPR1B_SPHNPISSVS_____
1        BMPR1B_HNPISSVS_______
2        BMPR1B_SVRCSSMS_______
3        BMPR1B_GSPHNPISSVS____
4        BMPR1B_GSPSVRCSSMS____
                  ...          
26413      MST3_DDSTLSESANQVFLG
26414      MST3_ALVKMLGSPVDSVLF
26415      MST3_QQEGGFASPRTGKGK
26416     CK1G3_PPHHGVQTPVQ____
26417     MEKK4_MEELKETTNSSEI__
Length: 26418, dtype: object

In [23]:
nwk

,substrate,substrate_id,#substrate,position,id,networkin_score,tree,netphorest_group,netphorest_score,string_identifier,string_score,substrate_name,sequence,string_path,kinase
0,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCalpha,1.0071,KIN,PKC_group,0.1576,ENSP00000282641,0.1738,A1CF,REEILsEMKKV,"ENSP00000284384, 0.7304 ENSP00000385269, 0.675...",PKCA
1,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCbeta,1.3889,KIN,PKC_group,0.1576,ENSP00000282641,0.1232,A1CF,REEILsEMKKV,"ENSP00000305355, 0.7328 ENSP00000348986, 0.627...",PKCB
2,A1CF,ENSP00000363105,A1CF (ENSP00000363105),154,PKCzeta,0.5163,KIN,PKC_group,0.1576,ENSP00000282641,0.1685,A1CF,REEILsEMKKV,"ENSP00000367830, 0.6584 ENSP00000226574, 0.727...",PKCZ
3,A1CF,ENSP00000363105,A1CF (ENSP00000363105),369,PKCalpha,1.7569,KIN,PKC_group,0.3374,ENSP00000282641,0.1738,A1CF,TKGHLsNRAII,"ENSP00000284384, 0.7304 ENSP00000385269, 0.675...",PKCA
4,A1CF,ENSP00000363105,A1CF (ENSP00000363105),369,PKCbeta,2.4948,KIN,PKC_group,0.3374,ENSP00000282641,0.1232,A1CF,TKGHLsNRAII,"ENSP00000305355, 0.7328 ENSP00000348986, 0.627...",PKCB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459743,ZZZ3,ENSP00000359837,ZZZ3 (ENSP00000359837),135,CK1alpha,1.3564,KIN,CK1_group,0.1338,ENSP00000359837,0.1079,ZZZ3,SSEEDsPIKSD,"ENSP00000261798, 0.54 ENSP00000363559, 0.2776 ...",CK1A
459744,ZZZ3,ENSP00000359837,ZZZ3 (ENSP00000359837),135,MAPK3,1.4275,KIN,MAPK3_MAPK1_MAPK7_NLK_group,0.1783,ENSP00000359837,0.1328,ZZZ3,SSEEDsPIKSD,"ENSP00000263025, 0.54 ENSP00000363559, 0.3416 ...",MAPK3
459745,ZZZ3,ENSP00000359837,ZZZ3 (ENSP00000359837),474,CK1alpha,1.3564,KIN,CK1_group,0.1249,ENSP00000359837,0.1079,ZZZ3,AKESAsQHITE,"ENSP00000261798, 0.54 ENSP00000363559, 0.2776 ...",CK1A
459746,ZZZ3,ENSP00000359837,ZZZ3 (ENSP00000359837),474,PAK1,0.7524,KIN,PAK_group,0.0438,ENSP00000359837,0.1020,ZZZ3,AKESAsQHITE,"ENSP00000278568, 0.54 ENSP00000363559, 0.2624 ...",PAK1


In [10]:
df_test1

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family,probs
0,FAK,Q05397,FAK,Q05397,IELGRCIGEGQFGDVHQGIYMSPENPALAVAIKTCKNCTSDSVREK...,IIDEEDTYTMPSTRD,Y407,447888.0,human,human,1,263.0,"[[[1.3951472984698037, 1.3951472984698037, 0.8...",ENSP00000431023,Fak,0.632388
1,ABL,P00519,Rb,P06400,ITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEA...,RIPGGNIYISPLKSP,Y805,456752.0,human,human,1,263.0,"[[[0.9656054162575665, 1.3951472984698037, 0.7...",ENSP00000496005,Abl,0.639248
2,MELK,Q14680,MELK,Q14680,YELHETIGTGGFAKVKLACHILTGEMVAIKIMDKNTLGSDLPRIKT...,SPERRCRSVELDLNQ,S505,2861409.0,human,human,1,263.0,"[[[1.0232665395472176, 0.8860339595928756, 0.8...",ENSP00000443550,CAMKL,0.774461
3,PKACA,P17612,EMSY,Q7Z589,FERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQEHTL...,KPRKRRRTNSSSSSP,T207,1210942.0,human,human,1,263.0,"[[[0.9762857097579093, 0.8702280284582515, 0.9...",ENSP00000436307,PKA,0.960219
4,SRC,P12931,tensin 3,Q68CZ2,0RLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQ...,QDTSKFWYKADISRE,Y1173,9824600.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000407464,Src,0.472669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,AMPKA1,Q13131,NaN,NaN,00LGDTLGVGTFGKVKVGKHELTGHKVAVKILNRQKIRSLDVGKIR...,AQVSRRISFSASHRL,NaN,NaN,NaN,human,0,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000434688,CAMKL,0.651911
3996,KSR2,Q6VAB6,NaN,NaN,LEIGELIGKGRFGQVYHGRWHGEVAIRLIDERDNEDQLKAFKREVM...,EAETLAETEPERHLG,NaN,NaN,NaN,human,0,NaN,"[[[0.9841273200552851, 1.3008267189517242, 0.9...",ENSP00000429729,RAF,0.684888
3997,MAPKAPK2,P49137,NaN,NaN,000SQVLGLGINGKVLQIFNKRTQEKFALKMLQDCPKARREVELHW...,LVDSVAKTMDAGCKP,NaN,NaN,NaN,human,0,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000486923,MAPKAPK,0.675056
3998,PKACA,P17612,NaN,NaN,FERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQEHTL...,EPEPENDYEDVEEMD,NaN,NaN,NaN,human,0,NaN,"[[[0.8798533791446438, 0.8702280284582515, 0.8...",ENSP00000419614,PKA,0.015924


In [34]:
df = pd.read_pickle("/home/hb/python/phospho/data/after1227_new_data/syt_false_dataset/1228_syt1_mtx_ensembl.pickle")

In [9]:
iloc_index = int(len(df)/2)
df_true = df.iloc[:iloc_index]


NameError: name 'df' is not defined

In [ ]:
df_true_ = df_true[~df_true['ensemble_id'].isin([0, ""])]

In [44]:
0 in df_true_.ensemble_id.unique(), "" in df_true_.ensemble_id.unique()

(False, False)

In [45]:
df_true_.ensemble_id.unique()

array(['ENSP00000426568', 'ENSP00000349282', 'ENSP00000394011', ...,
       'ENSMUSP00000112963', 'ENSP00000516040', 'ENSP00000443961'],
      dtype=object)

In [48]:
df_true_

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id
0,BMPR1B,O00238,SMAD1,Q15797,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,SPHNPISSVS_____,S463,448362.0,human,human,1,263.0,"[[[1.0253151205244289, 0.7268755493063382, 1.0...",ENSP00000426568
1,BMPR1B,O00238,SMAD1,Q15797,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,HNPISSVS_______,S465,448363.0,human,human,1,263.0,"[[[1.0273678027634894, 0.9579113900670306, 0.7...",ENSP00000426568
2,BMPR1B,O00238,SMAD2,Q15796,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,SVRCSSMS_______,S467,447495.0,human,human,1,263.0,"[[[1.0253151205244289, 1.3391030176392937, 0.9...",ENSP00000349282
3,BMPR1B,O00238,SMAD1,Q15797,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,GSPHNPISSVS____,S462,448361.0,human,human,1,263.0,"[[[0.9550419621907147, 1.0253151205244289, 0.7...",ENSP00000426568
4,BMPR1B,O00238,SMAD2,Q15796,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,GSPSVRCSSMS____,S464,447567.0,human,human,1,263.0,"[[[0.9550419621907147, 1.0253151205244289, 0.7...",ENSP00000349282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13202,MST3,Q9Y6E0,RAB8A,P61006,FTKLEKIGKGSFGEVFKGIDNRTQKVVAIKIIDLEEAEDEIEDIQQ...,AGQERFRTITTAYYR,T72,4709152.0,human,human,1,263.0,"[[[1.1151623503414478, 1.0191816486174081, 0.9...",ENSP00000467501
13203,MST3,Q9Y6E0,KEPI,Q8TAE6,FTKLEKIGKGSFGEVFKGIDNRTQKVVAIKIIDLEEAEDEIEDIQQ...,RHQQGKVTVKYDRKE,T73,10296602.0,human,human,1,263.0,"[[[0.9417645335842487, 1.060775240740159, 0.94...",ENSP00000355260
13204,MST3,Q9Y6E0,TAO1,Q7L7X3,FTKLEKIGKGSFGEVFKGIDNRTQKVVAIKIIDLEEAEDEIEDIQQ...,RNREHFATIRTASLV,T440,23150854.0,human,human,1,263.0,"[[[0.9417645335842487, 0.9617507091463667, 0.9...",ENSP00000516040
13205,MST3,Q9Y6E0,NDR2,Q9Y2H1,FTKLEKIGKGSFGEVFKGIDNRTQKVVAIKIIDLEEAEDEIEDIQQ...,DWVFLNYTYKRFEGL,T442,451133.0,human,human,1,263.0,"[[[0.8607079764250578, 1.1629966580818203, 1.1...",ENSP00000443961


In [52]:
df_true

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id
0,BMPR1B,O00238,SMAD1,Q15797,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,SPHNPISSVS_____,S463,448362.0,human,human,1,263.0,"[[[1.0253151205244289, 0.7268755493063382, 1.0...",ENSP00000426568
1,BMPR1B,O00238,SMAD1,Q15797,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,HNPISSVS_______,S465,448363.0,human,human,1,263.0,"[[[1.0273678027634894, 0.9579113900670306, 0.7...",ENSP00000426568
2,BMPR1B,O00238,SMAD2,Q15796,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,SVRCSSMS_______,S467,447495.0,human,human,1,263.0,"[[[1.0253151205244289, 1.3391030176392937, 0.9...",ENSP00000349282
3,BMPR1B,O00238,SMAD1,Q15797,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,GSPHNPISSVS____,S462,448361.0,human,human,1,263.0,"[[[0.9550419621907147, 1.0253151205244289, 0.7...",ENSP00000426568
4,BMPR1B,O00238,SMAD2,Q15796,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,GSPSVRCSSMS____,S464,447567.0,human,human,1,263.0,"[[[0.9550419621907147, 1.0253151205244289, 0.7...",ENSP00000349282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13204,MST3,Q9Y6E0,TAO1,Q7L7X3,FTKLEKIGKGSFGEVFKGIDNRTQKVVAIKIIDLEEAEDEIEDIQQ...,RNREHFATIRTASLV,T440,23150854.0,human,human,1,263.0,"[[[0.9417645335842487, 0.9617507091463667, 0.9...",ENSP00000516040
13205,MST3,Q9Y6E0,NDR2,Q9Y2H1,FTKLEKIGKGSFGEVFKGIDNRTQKVVAIKIIDLEEAEDEIEDIQQ...,DWVFLNYTYKRFEGL,T442,451133.0,human,human,1,263.0,"[[[0.8607079764250578, 1.1629966580818203, 1.1...",ENSP00000443961
13206,MST3,Q9Y6E0,MST3,Q9Y6E0,FTKLEKIGKGSFGEVFKGIDNRTQKVVAIKIIDLEEAEDEIEDIQQ...,DTQIKRNTFVGTPFW,T190,468211.0,human,human,1,263.0,"[[[0.8607079764250578, 1.0335505392413056, 0.9...",ENSP00000380651
13207,CK1G3,Q9Y6M4,LYN,P07948,0RVGKKIGCGNFGELRLGKNLYTNEYVAIKLEPMKSRAPQLHLEYR...,SKGKDSLSDDGVDLK,S13,1983291.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",


In [50]:
df_test = df_test[df_test['subseq'].isin(df_true_.subseq)]

In [51]:
df_test

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
551,P38D,O15264,DLG1,Q62696,000PTHVGSGAYGSVCSAIDKRSGEKVAIKKLSRPFQSEIFAKRAY...,FVSHSHISPIKPTEA,S158,2024309.0,human,rat,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,MAPK
553,P38D,O15264,DLG1,Q62696,000PTHVGSGAYGSVCSAIDKRSGEKVAIKKLSRPFQSEIFAKRAY...,VNTDSLETPTYVNGT,T209,35547401.0,human,rat,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,MAPK
739,ROCK2,O75116,MYPT1,Q10728,YDVVKVIGRGAFGEVQLVRHKASQKVYAMKLLSKFEMIKRSDAFFW...,RPREKRRSTGVSFWT,S854,451999.0,human,rat,1,263.0,"[[[1.0618365465453596, 0.8860339595928756, 1.0...",0,DMPK
741,ROCK2,O75116,CRMP-2,Q16555,YDVVKVIGRGAFGEVQLVRHKASQKVYAMKLLSKFEMIKRSDAFFW...,DNIPRRTTQRIVAPP,T555,448084.0,human,human,1,263.0,"[[[0.8478937040879159, 1.003004504503377, 1.18...",,DMPK
743,ROCK2,O75116,SNAP-25,P60879,YDVVKVIGRGAFGEVQLVRHKASQKVYAMKLLSKFEMIKRSDAFFW...,GGFIRRVTNDARENE,T138,449875.0,human,mouse,1,263.0,"[[[0.981179362242806, 0.981179362242806, 1.210...",,DMPK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25550,PINK1,Q9BXM7,NaN,NaN,000QLAHPNIIRVLRAFTSGRTLFLVMKNYPCTLRQYLCVNTPSPR...,ARQSRRSTQGVTLTD,NaN,NaN,NaN,NaN,0,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,NKF2
25648,HIPK2,Q9H2X6,NaN,NaN,YEVLEFLGRGTFGQVVKCWKRGTNEIVAIKILKNHPSYARQGQIEV...,IRVDTPHYPRWITKE,NaN,NaN,NaN,NaN,0,NaN,"[[[1.187677833213906, 1.0618365465453596, 1.16...",0,DYRK
26049,TLK1,Q9UKI8,NaN,NaN,000LHLLGRGGFSEVYKAFDLYEQRYAAVKIHQLNKRDEKKEKHAC...,AVLADVSYLMAMEKS,NaN,NaN,NaN,NaN,0,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",,TLK
26295,CAMK2A,Q9UQM7,NaN,NaN,YQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLE...,PVVSGDTSPRHLSNV,NaN,NaN,NaN,NaN,0,NaN,"[[[0.8860339595928756, 1.1653249789427378, 1.1...",0,CAMK2


In [21]:
ensembl_dict = {key: value for key, value in zip(df_true_.substrate, df_true_.ensemble_id,)}

In [47]:
subseq_dict = {key: value for key, value in zip(df_true_.subseq, df_true_.sub_id)}
df_test['sub_id'] = df_test['subseq'].apply(lambda x: subseq_dict[x])

KeyError: 'NNLRRSNSTTQVSQP'

In [30]:
df_test_ = df_test[df_test.substrate.isin(ensembl_dict.keys())]

In [31]:
df_test_

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
22,PLK4,O00444,PPP1R16B,Q96T49,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,KKVSFEASVALLEAS,S69,17549547.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",,PLK
36,CRIK,O14578,MRLC1,Q64122,0EVRSLVGCGHFAEVQVVREKATGDIYAMKVMKKKALLAQEQSFFE...,KRPQSATSNVFAMFD,S20,1863500.0,human,rat,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,DMPK
38,CRIK,O14578,MRLC1,Q64122,0EVRSLVGCGHFAEVQVVREKATGDIYAMKVMKKKALLAQEQSFFE...,KKRPQSATSNVFAMF,T19,1863501.0,human,rat,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,DMPK
274,IKKB,O14920,IkB-alpha,Q9Z1E3,0EMKERLGTGGFGNVIRWHNQETGEQIAIKQCRQELSPRNRERWCL...,LVDDRHDSGLDSMKD,S32,448203.0,human,mouse,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",,IKK
283,IKKB,O14920,PIK3R1,P26450,0EMKERLGTGGFGNVIRWHNQETGEQIAIKQCRQELSPRNRERWCL...,EPYNLYSSLKELVLH,S690,25018700.0,human,mouse,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",,IKK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26310,CAMK2A,Q9UQM7,ID2,NaN,YQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLE...,LGISRSKTPVDDPMS,NaN,NaN,NaN,NaN,0,NaN,"[[[1.152576648537005, 0.981179362242806, 1.187...",,CAMK2
26316,CAMK2A,Q9UQM7,NCCT,NaN,YQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLE...,SSTFCMRTFGYNTID,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0232665395472176, 1.0232665395472176, 1.0...",,CAMK2
26330,CAMK2A,Q9UQM7,HMGN1,NaN,YQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLE...,KNEESPASDEAEEKE,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0130848673598092, 1.003004504503377, 0.86...",0,CAMK2
26333,CAMK2A,Q9UQM7,SKP2,NaN,YQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLE...,ATLLSSCSRLDELNL,NaN,NaN,NaN,NaN,0,NaN,"[[[1.1241194296905368, 1.0273678027634894, 1.1...",,CAMK2


In [32]:
df_test_['ensemble_id'] = df_test_['substrate'].apply(lambda x: ensembl_dict[x])

/tmp/ipykernel_3164597/2471985305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_['ensemble_id'] = df_test_['substrate'].apply(lambda x: ensembl_dict[x])


In [33]:
df_test_

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
22,PLK4,O00444,PPP1R16B,Q96T49,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,KKVSFEASVALLEAS,S69,17549547.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSBTAP00000000858,PLK
36,CRIK,O14578,MRLC1,Q64122,0EVRSLVGCGHFAEVQVVREKATGDIYAMKVMKKKALLAQEQSFFE...,KRPQSATSNVFAMFD,S20,1863500.0,human,rat,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000279022,DMPK
38,CRIK,O14578,MRLC1,Q64122,0EVRSLVGCGHFAEVQVVREKATGDIYAMKVMKKKALLAQEQSFFE...,KKRPQSATSNVFAMF,T19,1863501.0,human,rat,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000279022,DMPK
274,IKKB,O14920,IkB-alpha,Q9Z1E3,0EMKERLGTGGFGNVIRWHNQETGEQIAIKQCRQELSPRNRERWCL...,LVDDRHDSGLDSMKD,S32,448203.0,human,mouse,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000513488,IKK
283,IKKB,O14920,PIK3R1,P26450,0EMKERLGTGGFGNVIRWHNQETGEQIAIKQCRQELSPRNRERWCL...,EPYNLYSSLKELVLH,S690,25018700.0,human,mouse,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000513328,IKK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26310,CAMK2A,Q9UQM7,ID2,NaN,YQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLE...,LGISRSKTPVDDPMS,NaN,NaN,NaN,NaN,0,NaN,"[[[1.152576648537005, 0.981179362242806, 1.187...",ENSMUSP00000020974,CAMK2
26316,CAMK2A,Q9UQM7,NCCT,NaN,YQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLE...,SSTFCMRTFGYNTID,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0232665395472176, 1.0232665395472176, 1.0...",ENSMUSP00000148537,CAMK2
26330,CAMK2A,Q9UQM7,HMGN1,NaN,YQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLE...,KNEESPASDEAEEKE,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0130848673598092, 1.003004504503377, 0.86...",ENSP00000395787,CAMK2
26333,CAMK2A,Q9UQM7,SKP2,NaN,YQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLE...,ATLLSSCSRLDELNL,NaN,NaN,NaN,NaN,0,NaN,"[[[1.1241194296905368, 1.0273678027634894, 1.1...",ENSP00000503224,CAMK2


In [61]:
df_train = pd.read_pickle("/home/hb/python/phospho/data/after1227_new_data/syt_false_dataset/train_test/train3.pickle")

In [62]:
dff = pd.concat([df_train, df_test], ignore_index=True,)
dff_ = dff.dropna(subset=['substrate'], )
dff_.reset_index(drop=True, inplace=True)
dff_

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
0,P38A,Q16539,STAT3,P40763,YQNLSPVGSGAYGSVCAAFDTKTGLRVAVKKLSRPFQSIIHAKRTY...,NTIDLPMSPRTLDSL,S727,448191.0,human,human,1,263.0,"[[[1.003004504503377, 1.0273678027634894, 1.18...",ENSP00000384943,MAPK
1,CK2A1,P68400,HES6,Q96HZ4,YQLVRKLGRGKYSEVFEAINITNNEKVVVKILKPVKKKKIKREIKI...,GPGDDLCSDLEEAPE,S183,455670.0,human,human,1,263.0,"[[[0.981179362242806, 0.8860339595928756, 0.98...",ENSP00000390870,CK2
2,AURB,Q96GD4,KIF2C,Q99661,FEIGRPLGKGKFGNVYLAREKKSHFIVALKVLFKSQIEKEGVHQLR...,KESLRSRSTRMSTVS,S111,25159900.0,human,human,1,263.0,"[[[0.9762857097579093, 0.8798533791446438, 1.0...",ENSP00000402172,AUR
3,ULK1,O75385,ATG9A,Q7Z3C6,0SRKDLIGHGAFAVVFKGRHREKHDEVAVKCINKKNLAKSQTLLGK...,EYQRLEASYSDSPPG,S14,3203970.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000394345,ULK
4,MST2,Q13188,SAV1,Q9H4B6,0DVLEKLGEGSYGSVFKAIHKESGQVVAIKQVPVESDLQEIIKEIS...,YRHPCAPSVPRYDQP,S269,21507301.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000451596,STE20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13204,DYRK1B,Q9Y463,ID2,Q02363,YEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIEL...,LGISRSKTPVDDPMS,T27,4706880.0,human,human,1,263.0,"[[[1.152576648537005, 0.981179362242806, 1.187...",,DYRK
13205,MRCKB,Q9Y5S2,SIAH2,O43255,FEIIKVIGRGAFGEVAVVKMKNTERIYAMKILNKWEMLKRAEACFR...,RRLTWEATPRSIHDG,T279,1872043500.0,human,human,1,263.0,"[[[0.9417645335842487, 0.9417645335842487, 1.0...",,DMPK
13206,MRCKB,Q9Y5S2,SIAH2,O43255,FEIIKVIGRGAFGEVAVVKMKNTERIYAMKILNKWEMLKRAEACFR...,__MSRPSSTGPSANK,S6,25172963.0,human,human,1,263.0,"[[[0.0, 0.0, 1.1653249789427378, 1.07896257415...",,DMPK
13207,CK1G3,Q9Y6M4,LYN,P07948,0RVGKKIGCGNFGELRLGKNLYTNEYVAIKLEPMKSRAPQLHLEYR...,SKGKDSLSDDGVDLK,S13,1983291.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",,CK1


In [8]:
sub_subseq_dict = {}
for i in range(len(dff_)):
    sub_subseq_dict[dff_.subseq[i]] = dff_.substrate[i]

In [84]:
df_test.reset_index(drop=True, inplace=True)

for idx in range(len(df_test)):
    if df_test.subseq[idx] in sub_subseq_dict.keys():
        df_test.substrate[idx] = sub_subseq_dict[df_test.subseq[idx]]

t = [">" + x + "\n" + y for x, y in zip(df_test['substrate'], df_test['subseq'])]
print("\n".join(t))

>CEP131
NNLRRSNSTTQVSQP
>PCM-1
RRQAESLSLTREVSQ
>PPP1R16B
KKVSFEASVALLEAS
>CEP131
SPAGVDLSLTGLPPP
>CEP131
PLKSSNQTAPSLNNI
>MRLC1
KRPQSATSNVFAMFD
>MRLC1
KKRPQSATSNVFAMF
>JNK1
TSFMMTPYVVTRYYR
>CDC25C
SGLYRSPSMPENLNR
>RALY
PLVRRVKTNVPVKLF
>KI-67
RSLRKADTEEEFLAF
>FANCE
LFLGRILSLTSSASR
>CK1D
RGLPSTASGRLRGTQ
>CK1D
MERERKVSMRLHRGA
>CK1D
QDTSRMSTSQIPGRV
>CK1D
PATRGLPSTASGRLR
>CLK3
HHCKRYRSPEPDPYL
>CEP170B
QGLPVPGSPGGQKWV
>LMO7
GIMRRGESLDNLDSP
>ING1 iso2
LGDTAGNSGKAGADR
>vigilin
MSSVAVLTQESFAEH
>WIZ
SLEARSPSDLHISPL
>PRRC2C
QPRPAVLSGYFKQFQ
>E2F8
PSFTRHPSLIKLVKS
>BLM
DQNKHTASINDLERE
>LAMC1
VTLNRLNTFGDEVFN
>YB-1
GFINRNDTKEDVFVH
>SETMAR
LYDNRRRSAQWLDQE
>FANCE
LGLLRLCTWLLALSP
>CAAP1
EGDNGMDSDMEEEAD
>ASF1A
KLEDAESSNPNLQSL
>SNCB
EDPPQEEYQEYEPEA
>CCDC6
QPVSAPPSPRDISME
>Hck
RVIEDNEYTAREGAK
>MCM3
KTIERRYSDLTTLVA
>BLM
SHFVRVSTAQKSKKG
>LATS2
PVPSRTNSFNSHQPR
>CCDC6
EKLDQPVSAPPSPRD
>LATS2
GSHVRQDSMEPSDLW
>BLM
HLQKSFVSSNWAETP
>SKAR
ARQKIGLSDARLKLG
>FYTTD1
KVQAQLNTEQLLDDV
>BLM
LKHERFQSLSFPHTK
>NHS
RKLRRRKTISGIP

In [63]:
dff_dropna = dff_.dropna(subset=['ensemble_id'])

In [71]:
ensemble_id_list = list(dff_dropna.ensemble_id.unique())[:-3]

In [86]:
ensembl_id_dict = {}
for i in range(len(dff_)):
    if dff_.ensemble_id[i] in ensemble_id_list:
        ensembl_id_dict[dff_.substrate[i]] = dff_.ensemble_id[i]

In [91]:
df_test['subseq_'] = df_test['subseq'].apply(lambda x: x[:7] + x[7].lower() + x[8:])

In [99]:
for i in range(len(df_test)):
    if df_test.substrate[i] in ensembl_id_dict.keys():
        df_test.ensemble_id[i] = ensembl_id_dict[df_test.substrate[i]]

0.8678644618100564

In [15]:
def set_pred(model_name, df, type):
    probs = ensemble_5fold(model_name, df, type)
    df[[f'{type}_probs']] = probs
    auc = sklearn.metrics.roc_auc_score(df['answer'], df[f'{type}_probs'])
    return probs, auc

In [56]:
df_train

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
0,GSK3B,P49841,NaN,NaN,YTDTKVIGNGSFGVVYQAKLCDSGELVAIKKVLQDKRFKNRELQIM...,EIEILLRYGQHPNII,NaN,NaN,NaN,NaN,0,NaN,"[[[0.8650222931107413, 1.187677833213906, 0.86...",ENSP00000388512,GSK
1,P38A,Q16539,STAT3,P40763,YQNLSPVGSGAYGSVCAAFDTKTGLRVAVKKLSRPFQSIIHAKRTY...,NTIDLPMSPRTLDSL,S727,448191.0,human,human,1,263.0,"[[[1.003004504503377, 1.0273678027634894, 1.18...",ENSP00000384943,MAPK
2,CDK1,P06493,NaN,NaN,YTKIEKIGEGTYGVVYKGRHKTTGQVVAMKKIRLESEEEGVPSTAI...,NNGGNEDSDENEDEE,NaN,NaN,NaN,NaN,0,NaN,"[[[1.003004504503377, 1.003004504503377, 0.981...",ENSBTAP00000000595,CDK
3,ERK2,P28482,NaN,NaN,YTNLSYIGEGAYGMVCSAYDNVNKVRVAIKKISPFEHQTYCQRTLR...,QTDPSGTYHAWKANA,NaN,NaN,NaN,NaN,0,NaN,"[[[1.003004504503377, 1.0273678027634894, 0.84...",ENSP00000359895,MAPK
4,PKACA,P17612,NaN,NaN,FERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQEHTL...,PPEQETFYEQPPLVQ,NaN,NaN,NaN,NaN,0,NaN,"[[[0.8702280284582515, 0.8702280284582515, 0.8...",ENSP00000397855,PKA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21156,CDK1,P06493,NEDD1,Q8NHV4,YTKIEKIGEGTYGVVYKGRHKTTGQVVAMKKIRLESEEEGVPSTAI...,PPINGSSTPNPKIAS,T550,6225645.0,human,human,1,263.0,"[[[0.8860339595928756, 0.8860339595928756, 1.1...",ENSP00000407964,CDK
21157,CDK2,P24941,BCLAF1,Q9NYF8,FQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAI...,EEWDPEYTPKSKKYF,T840,455929.0,human,human,1,263.0,"[[[0.8798533791446438, 0.8798533791446438, 1.1...",ENSP00000487249,CDK
21158,CDK2,P24941,NaN,NaN,FQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAI...,SFSVKDPSPLYDMLR,NaN,NaN,NaN,NaN,0,NaN,"[[[1.078962574157284, 1.2969300866657718, 1.07...",ENSP00000395254,CDK
21159,ERK1,P27361,eplin,Q9UHB6,YTQLQYIGEGAYGMVSSAYDHVRKTRVAIKKISPFEHQTYCQRTLR...,ASSLSESSPPKAMKK,S374,452503.0,human,human,1,263.0,"[[[1.1241194296905368, 1.0232665395472176, 1.0...",ENSP00000449890,MAPK


In [15]:
df_test

,kinase,kin_id,substrate,sub_id,kinseq,subseq,answer,length,pair,matrix,family
0,BMPR1B,O00238,SMAD1,Q15797,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,HNPISSVS_______,1,263.0,O00238_Q15797,"[[[1.0273678027634894, 0.9579113900670306, 0.7...",STKR
1,BMPR1B,O00238,SMAD2,Q15796,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,GSPSVRCSSMS____,1,263.0,O00238_Q15796,"[[[0.9550419621907147, 1.0253151205244289, 0.7...",STKR
2,BMPR1B,O00238,SMAD1,Q15797,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,SPHNPISSVS_____,1,263.0,O00238_Q15797,"[[[1.0253151205244289, 0.7268755493063382, 1.0...",STKR
3,BMPR1B,O00238,SMAD2,Q15796,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,SPSVRCSSMS_____,1,263.0,O00238_Q15796,"[[[1.0253151205244289, 0.7268755493063382, 1.0...",STKR
4,BMPR1B,O00238,SMAD1,Q15797,IQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQ...,GSPHNPISSVS____,1,263.0,O00238_Q15797,"[[[0.9550419621907147, 1.0253151205244289, 0.7...",STKR
...,...,...,...,...,...,...,...,...,...,...,...
26521,RIPK3,Q9Y572,NaN,NaN,0ENQELVGKGGFGTVFRAQHRKWGYDVAVKIVNSKAISREVKAMAS...,SLSARKLSLQERPAG,0,NaN,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",RIPK
26522,RIPK3,Q9Y572,NaN,NaN,0ENQELVGKGGFGTVFRAQHRKWGYDVAVKIVNSKAISREVKAMAS...,SPVLAEDSEGEG___,0,NaN,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",RIPK
26523,RIPK3,Q9Y572,NaN,NaN,0ENQELVGKGGFGTVFRAQHRKWGYDVAVKIVNSKAISREVKAMAS...,GPGAELLTRSPSRSS,0,NaN,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",RIPK
26524,RIPK3,Q9Y572,NaN,NaN,0ENQELVGKGGFGTVFRAQHRKWGYDVAVKIVNSKAISREVKAMAS...,LEQLTGQTPVFSKAR,0,NaN,NaN,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",RIPK


In [28]:
random1_probs, random1_auc = set_pred(model_list_random1, df_test, 'matrix')

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: unknown format is not supported

In [15]:
train = pd.read_pickle("/home/hb/python/phospho/data/after1227_new_data/syt_false_dataset/train_test/train2.pickle")

In [16]:
test = pd.read_pickle("/home/hb/python/phospho/data/after1227_new_data/syt_false_dataset/train_test/test2.pickle")

In [17]:
train

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
0,ERK1,P27361,BIM,O43521,YTQLQYIGEGAYGMVSSAYDHVRKTRVAIKKISPFEHQTYCQRTLR...,GPLAPPASPGPFATR,S69,449330.0,human,human,1,263.0,"[[[0.981179362242806, 0.8860339595928756, 1.15...",ENSP00000389225,MAPK
1,GRK6,P43250,NHERF,O14745,0RQYRVLGKGGFGEVCACQVRATGKMYACKKLEKKRIKKRKGAMAL...,PALVRSASSDTSEEL,S290,450313.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000464117,GRK
2,ITK,Q08881,PLCG1,P19174,LTFVQEIGSGQFGLVHLGYWLNKDKVAIKTIREGAMSEEDFIEEAE...,EGRNPGFYVEANPMP,Y783,447667.0,human,human,1,263.0,"[[[0.9841273200552851, 0.9121051495450904, 1.0...",ENSP00000476454,Tec
3,PKACA,P17612,SFRS2,Q01130,FERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQEHTL...,GYGRRSRSPRRRRRS,S121,6422897.0,human,human,1,263.0,"[[[1.0191816486174081, 1.2104594520812995, 1.0...",ENSP00000350877,PKA
4,AURA,O14965,NaN,NaN,FEIGRPLGKGKFGNVYLAREKQSKFILALKVLFKAQLEKAGVHQLR...,VDSGNDIYGNPIKRI,NaN,NaN,NaN,NaN,0,NaN,"[[[1.1606729892090855, 0.8607079764250578, 1.0...",ENSP00000464930,AUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21147,CDK1,P06493,NEDD1,Q8NHV4,YTKIEKIGEGTYGVVYKGRHKTTGQVVAMKKIRLESEEEGVPSTAI...,PPINGSSTPNPKIAS,T550,6225645.0,human,human,1,263.0,"[[[0.8860339595928756, 0.8860339595928756, 1.1...",ENSP00000407964,CDK
21148,CDK2,P24941,BCLAF1,Q9NYF8,FQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAI...,EEWDPEYTPKSKKYF,T840,455929.0,human,human,1,263.0,"[[[0.8798533791446438, 0.8798533791446438, 1.1...",ENSP00000487249,CDK
21149,CDK2,P24941,NaN,NaN,FQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAI...,ASSSTSVTPDVSDNE,NaN,NaN,NaN,NaN,0,NaN,"[[[1.1151623503414478, 1.078962574157284, 1.07...",ENSP00000514759,CDK
21150,ERK1,P27361,eplin,Q9UHB6,YTQLQYIGEGAYGMVSSAYDHVRKTRVAIKKISPFEHQTYCQRTLR...,ASSLSESSPPKAMKK,S374,452503.0,human,human,1,263.0,"[[[1.1241194296905368, 1.0232665395472176, 1.0...",ENSP00000449890,MAPK


In [18]:
test

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
7,PLK4,O00444,CEP131,Q9UPN4,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,NNLRRSNSTTQVSQP,S78,1211822.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",,PLK
15,PLK4,O00444,PCM-1,Q15154,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,RRQAESLSLTREVSQ,S372,11536118.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,PLK
22,PLK4,O00444,PPP1R16B,Q96T49,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,KKVSFEASVALLEAS,S69,17549547.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",,PLK
25,PLK4,O00444,CEP131,Q9UPN4,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,SPAGVDLSLTGLPPP,S21,41451326.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",,PLK
27,PLK4,O00444,CEP131,Q9UPN4,0KVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMQRVQ...,PLKSSNQTAPSLNNI,T205,1868722300.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",,PLK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26387,DYRK1B,Q9Y463,NaN,NaN,YEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIEL...,GHRKYREYSPPYAPS,NaN,NaN,NaN,NaN,0,NaN,"[[[0.981179362242806, 1.0639623447280337, 1.06...",,DYRK
26389,DYRK1B,Q9Y463,NaN,NaN,YEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIEL...,SMPGNRLYVHPDSPN,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0232665395472176, 1.1936310931271388, 0.8...",,DYRK
26390,DYRK1B,Q9Y463,NaN,NaN,YEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIEL...,SVSGAAPYSSFMPPE,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0232665395472176, 1.1653249789427378, 1.0...",0,DYRK
26392,DYRK1B,Q9Y463,NaN,NaN,YEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIEL...,EEEGRDEYDEVAMPV,NaN,NaN,NaN,NaN,0,NaN,"[[[0.8650222931107413, 0.8650222931107413, 0.8...",,DYRK


In [21]:
t = pd.concat([train, test], ignore_index=True)


In [26]:
t[t.duplicated(subset=['kinseq', 'subseq'])]

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family


In [28]:
t

,kinase,kin_id,substrate,sub_id,kinseq,subseq,sub_mod_rsd,sub_position,kin_oragnism,sub_oragnism,answer,length,matrix,ensemble_id,family
0,ERK1,P27361,BIM,O43521,YTQLQYIGEGAYGMVSSAYDHVRKTRVAIKKISPFEHQTYCQRTLR...,GPLAPPASPGPFATR,S69,449330.0,human,human,1,263.0,"[[[0.981179362242806, 0.8860339595928756, 1.15...",ENSP00000389225,MAPK
1,GRK6,P43250,NHERF,O14745,0RQYRVLGKGGFGEVCACQVRATGKMYACKKLEKKRIKKRKGAMAL...,PALVRSASSDTSEEL,S290,450313.0,human,human,1,263.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",ENSP00000464117,GRK
2,ITK,Q08881,PLCG1,P19174,LTFVQEIGSGQFGLVHLGYWLNKDKVAIKTIREGAMSEEDFIEEAE...,EGRNPGFYVEANPMP,Y783,447667.0,human,human,1,263.0,"[[[0.9841273200552851, 0.9121051495450904, 1.0...",ENSP00000476454,Tec
3,PKACA,P17612,SFRS2,Q01130,FERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQEHTL...,GYGRRSRSPRRRRRS,S121,6422897.0,human,human,1,263.0,"[[[1.0191816486174081, 1.2104594520812995, 1.0...",ENSP00000350877,PKA
4,AURA,O14965,NaN,NaN,FEIGRPLGKGKFGNVYLAREKQSKFILALKVLFKAQLEKAGVHQLR...,VDSGNDIYGNPIKRI,NaN,NaN,NaN,NaN,0,NaN,"[[[1.1606729892090855, 0.8607079764250578, 1.0...",ENSP00000464930,AUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26413,DYRK1B,Q9Y463,NaN,NaN,YEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIEL...,GHRKYREYSPPYAPS,NaN,NaN,NaN,NaN,0,NaN,"[[[0.981179362242806, 1.0639623447280337, 1.06...",,DYRK
26414,DYRK1B,Q9Y463,NaN,NaN,YEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIEL...,SMPGNRLYVHPDSPN,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0232665395472176, 1.1936310931271388, 0.8...",,DYRK
26415,DYRK1B,Q9Y463,NaN,NaN,YEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIEL...,SVSGAAPYSSFMPPE,NaN,NaN,NaN,NaN,0,NaN,"[[[1.0232665395472176, 1.1653249789427378, 1.0...",0,DYRK
26416,DYRK1B,Q9Y463,NaN,NaN,YEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIEL...,EEEGRDEYDEVAMPV,NaN,NaN,NaN,NaN,0,NaN,"[[[0.8650222931107413, 0.8650222931107413, 0.8...",,DYRK


In [27]:
t[t.duplicated(subset=['matrix'])]

TypeError: unhashable type: 'numpy.ndarray'

## GPS

In [11]:
df = pd.read_csv("/home/hb/python/phospho/GPS6.0/20221220-201905-775/20221220-201905-775.gps", sep='\t')

In [12]:
df

ID  Position    Code Kinase  \
AML3 2  S AGC/DMPK/CRIK      ******DSEDYSLSE    0.4892  0.4021     --   
     6  S AGC/DMPK/CRIK      **DSEDYSLSEEGQE    0.4600  0.4021     --   
     8  S AGC/DMPK/CRIK      DSEDYSLSEEGQELS    0.4600  0.4021     --   
     15 S AGC/DMPK/CRIK      SEEGQELS*******    0.4892  0.4021     --   
     2  S AGC/DMPK/CRIK/CIT  ******DSEDYSLSE    0.4892  0.4021     --   
...                                      ...       ...     ...    ...   
     8  S Dual/TK/Abl        DSEDYSLSEEGQELS    0.3257  0.3237     --   
     2  S Dual/TK/Abl/ABL1   ******DSEDYSLSE    0.3308  0.2858     --   
     6  S Dual/TK/Abl/ABL1   **DSEDYSLSEEGQE    0.4532  0.2858     --   
     8  S Dual/TK/Abl/ABL1   DSEDYSLSEEGQELS    0.4532  0.2858     --   
     15 S Dual/TK/Abl/ABL1   SEEGQELS*******    0.3303  0.2858     --   

                                                                       Peptide  \
AML3 2  S AGC/DMPK/CRIK      *****************************DSEDYSLSEEGQELS**...   
     6  S AGC/DMPK/CRIK      *************************DSEDYSLSEEGQELS******...   
     8  S AGC/DMPK/CRIK      ***********************DSEDYSLSEEGQELS********...   
     15 S AGC/DMPK/CRIK      ****************DSEDYSLSEEGQELS***************...   
     2  S AGC/DMPK/CRIK/CIT  *****************************DSEDYSLSEEGQELS**...   
...                                                                        ...   
     8  S Dual/TK/Abl        ***********************DSEDYSLSEEGQELS********...   
     2  S Dual/TK/Abl/ABL1   *****************************DSEDYSLSEEGQELS**...   
     6  S Dual/TK/Abl/ABL1   *************************DSEDYSLSEEGQELS******...   
     8  S Dual/TK/Abl/ABL1   ***********************DSEDYSLSEEGQELS********...   
     15 S Dual/TK/Abl/ABL1   ****************DSEDYSLSEEGQELS***************...   

                             Score  \
AML3 2  S AGC/DMPK/CRIK      Pred.   
     6  S AGC/DMPK/CRIK      Pred.   
     8  S AGC/DMPK/CRIK      Pred.   
     15 S AGC/DMPK/CRIK      Pred.   
     2  S AGC/DMPK/CRIK/CIT  Pred.   
...                            ...   
     8  S Dual/TK/Abl        Pred.   
     2  S Dual/TK/Abl/ABL1   Pred.   
     6  S Dual/TK/Abl/ABL1   Pred.   
     8  S Dual/TK/Abl/ABL1   Pred.   
     15 S Dual/TK/Abl/ABL1   Pred.   

                                                                        Cutoff  
AML3 2  S AGC/DMPK/CRIK      <a href =https://thebiogrid.org/  target='_bla...  
     6  S AGC/DMPK/CRIK      <a href =https://thebiogrid.org/  target='_bla...  
     8  S AGC/DMPK/CRIK      <a href =https://thebiogrid.org/  target='_bla...  
     15 S AGC/DMPK/CRIK      <a href =https://thebiogrid.org/  target='_bla...  
     2  S AGC/DMPK/CRIK/CIT  <a href =https://thebiogrid.org/  target='_bla...  
...                                                                        ...  
     8  S Dual/TK/Abl        <a href =https://thebiogrid.org/  target='_bla...  
     2  S Dual/TK/Abl/ABL1   <a href =https://thebiogrid.org/  target='_bla...  
     6  S Dual/TK/Abl/ABL1   <a href =https://thebiogrid.org/  target='_bla...  
     8  S Dual/TK/Abl/ABL1   <a href =https://thebiogrid.org/  target='_bla...  
     15 S Dual/TK/Abl/ABL1   <a href =https://thebiogrid.org/  target='_bla...  

[561 rows x 7 columns]